In [1]:
import polars as pl
import pandas as pd
import geopandas as gpd
import datetime

import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

from src.procedures import fetch_last_data
from dotenv import dotenv_values

config = dotenv_values("../.env")

In [17]:
# Retrieves the most recently updated data from the database
value = 3

CONNECTION_URI = config.get("CONNECTION_URI")
query = f"""
    SELECT latitude, longitude, acq_date, acq_time, confidence, frp, brightness, second_adm, first_adm
    FROM processed_viirs 
    WHERE acq_date > CURRENT_DATE - INTERVAL '{value} day'"""

processed_viirs = fetch_last_data(query=query, uri_connection=CONNECTION_URI)
df_viirs = processed_viirs.to_pandas()

df_viirs = df_viirs.rename(columns={"frp":"Fire Radiative Power", "second_adm": "Kabupaten/Kota", "first_adm":"Provinsi",
                                  "acq_date":"Tanggal", "confidence":"Confidence", "brightness":"Brightness"})

df_viirs

,latitude,longitude,Tanggal,acq_time,Confidence,Fire Radiative Power,Brightness,Kabupaten/Kota,Provinsi
0,-8.90874,140.821320,2023-09-08,427,Nominal,5.28,337.829987,Merauke,Papua
1,-8.90812,140.824982,2023-09-08,427,Nominal,5.28,344.279999,Merauke,Papua
2,-8.47618,140.526978,2023-09-08,427,Nominal,4.16,331.250000,Merauke,Papua
3,-8.39021,140.970230,2023-09-08,427,Nominal,4.65,332.910004,Merauke,Papua
4,-10.19829,123.920174,2023-09-08,429,Nominal,5.70,341.130005,Kupang,Nusa Tenggara Timur
...,...,...,...,...,...,...,...,...,...
4317,-7.54137,110.445061,2023-09-09,1838,Nominal,5.43,347.279999,Magelang,Jawa Tengah
4318,-7.54104,110.442650,2023-09-09,1838,Nominal,5.43,332.399994,Magelang,Jawa Tengah
4319,-7.33304,111.061813,2023-09-09,1838,Nominal,0.40,300.489990,Sragen,Jawa Tengah
4320,-7.07293,108.995522,2023-09-09,1838,Nominal,1.59,308.339996,Tegal,Jawa Tengah


In [32]:
df_viirs["Tanggal"] = df_viirs["Tanggal"].astype(str)

hover_dict = {"latitude":False, "longitude":False, "Tanggal":True, "acq_time":False, 
              "Confidence":True,"Fire Radiative Power":True, "Kabupaten/Kota":True, 
              "Provinsi":False, "Brightness":True}


map_fig = px.density_mapbox(df_viirs, lat="latitude", lon="longitude", z="Brightness",
                             radius=1, hover_name="Provinsi",
                             hover_data=hover_dict,
                            center=dict(lat=-2.5, lon=118), zoom=3.7, color_continuous_scale="matter_r", 
                            mapbox_style="open-street-map", template="plotly_white"
                            )

map_fig.update_layout(autosize=False,width=1200,height=400)
map_fig.update_layout(margin={"r":1,"t":1,"l":1,"b":1})
# map_fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)',})
map_fig.update_coloraxes(showscale=True, colorbar=dict(len=0.75, title="Brightness", thickness=15))

map_fig
